In [3]:
import pandas as pd
import numpy as np


In [4]:
TRAIN_DATA_PATH = r"C:\Users\ammym\OneDrive\Desktop\CDC\train_data.csv"
TEST_DATA_PATH = r"C:\Users\ammym\OneDrive\Desktop\CDC\test_data.csv"

df_train = pd.read_csv(TRAIN_DATA_PATH)
df_test = pd.read_csv(TEST_DATA_PATH)


print("Original train dataset shape:", df_train.shape)

Original train dataset shape: (16209, 21)


In [5]:
df_train["id"] = df_train["id"].astype(str)

df_train["date"] = pd.to_datetime(df_train["date"])
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16209 entries, 0 to 16208
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             16209 non-null  object        
 1   date           16209 non-null  datetime64[ns]
 2   price          16209 non-null  int64         
 3   bedrooms       16209 non-null  int64         
 4   bathrooms      16209 non-null  float64       
 5   sqft_living    16209 non-null  int64         
 6   sqft_lot       16209 non-null  int64         
 7   floors         16209 non-null  float64       
 8   waterfront     16209 non-null  int64         
 9   view           16209 non-null  int64         
 10  condition      16209 non-null  int64         
 11  grade          16209 non-null  int64         
 12  sqft_above     16209 non-null  int64         
 13  sqft_basement  16209 non-null  int64         
 14  yr_built       16209 non-null  int64         
 15  yr_renovated   1620

In [6]:
df_train = df_train.sort_values("date")
df_train = df_train.groupby("id", as_index = False).last()

In [7]:
df_train["log_price"] = np.log1p(df_train["price"])

In [8]:
df_train["above_ground_ratio"] = df_train["sqft_above"] / (df_train["sqft_living"] + 1)
df_train["bath_per_bed"] = df_train["bathrooms"] / (df_train["bedrooms"] + 1)
df_train["living_to_lot_ratio"] = df_train["sqft_living"] / (df_train["sqft_lot"] + 1)


In [9]:
df_train["relative_living_size"] = (
    df_train["sqft_living"] / (df_train["sqft_living15"] + 1)
)

df_train["relative_lot_size"] = (
    df_train["sqft_lot"] / (df_train["sqft_lot15"] + 1)
)
df_train["neighborhood_density"] = (
    df_train["sqft_lot15"] / (df_train["sqft_living15"] + 1)
)

In [10]:
lat_center = df_train["lat"].mean()
lon_center = df_train["long"].mean()

df_train["distance_from_center"] = np.sqrt(
    (df_train["lat"] - lat_center) ** 2 +
    (df_train["long"] - lon_center) ** 2
)

df_train["waterfront_view"] = df_train["waterfront"] * df_train["view"]

In [11]:
# Effective age of the property
df_train["house_age"] = 2015 - df_train["yr_built"]

# Renovation indicator
df_train["is_renovated"] = (df_train["yr_renovated"] > 0).astype(int)

# Years since renovation (if any)
df_train["years_since_reno"] = np.where(
    df_train["yr_renovated"] > 0,
    2015 - df_train["yr_renovated"],
    0
)

df_train["year_sale"] = df_train["date"].dt.year
df_train["month_sale"] = df_train["date"].dt.month
df_train["quarter_sale"] = df_train["date"].dt.quarter

In [12]:
df_train.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'log_price',
       'above_ground_ratio', 'bath_per_bed', 'living_to_lot_ratio',
       'relative_living_size', 'relative_lot_size', 'neighborhood_density',
       'distance_from_center', 'waterfront_view', 'house_age', 'is_renovated',
       'years_since_reno', 'year_sale', 'month_sale', 'quarter_sale'],
      dtype='object')

In [13]:
FEATURE_COLS = [
       'id','bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15', 'log_price',
       'above_ground_ratio', 'bath_per_bed', 'living_to_lot_ratio',
       'relative_living_size', 'relative_lot_size', 'neighborhood_density',
       'distance_from_center', 'waterfront_view', 'house_age', 'is_renovated',
       'years_since_reno', 'year_sale', 'month_sale', 'quarter_sale'
]

df_train_preprocessed = df_train[FEATURE_COLS]
df_train_preprocessed.shape

(16110, 34)

In [14]:
df_train_preprocessed.to_csv(r"C:\Users\ammym\OneDrive\Desktop\CDC\train_preprocessed.csv", index = False)

In [15]:
df_test["id"] = df_test["id"].astype(str)

df_test["date"] = pd.to_datetime(df_test["date"])
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5404 entries, 0 to 5403
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             5404 non-null   object        
 1   date           5404 non-null   datetime64[ns]
 2   bedrooms       5404 non-null   int64         
 3   bathrooms      5404 non-null   float64       
 4   sqft_living    5404 non-null   int64         
 5   sqft_lot       5404 non-null   int64         
 6   floors         5404 non-null   float64       
 7   waterfront     5404 non-null   int64         
 8   view           5404 non-null   int64         
 9   condition      5404 non-null   int64         
 10  grade          5404 non-null   int64         
 11  sqft_above     5404 non-null   int64         
 12  sqft_basement  5404 non-null   int64         
 13  yr_built       5404 non-null   int64         
 14  yr_renovated   5404 non-null   int64         
 15  zipcode        5404 n

In [16]:
df_test["above_ground_ratio"] = df_test["sqft_above"] / (df_test["sqft_living"] + 1)
df_test["bath_per_bed"] = df_test["bathrooms"] / (df_test["bedrooms"] + 1)
df_test["living_to_lot_ratio"] = df_test["sqft_living"] / (df_test["sqft_lot"] + 1)


In [17]:
df_test["relative_living_size"] = (
    df_test["sqft_living"] / (df_test["sqft_living15"] + 1)
)

df_test["relative_lot_size"] = (
    df_test["sqft_lot"] / (df_test["sqft_lot15"] + 1)
)
df_test["neighborhood_density"] = (
    df_test["sqft_lot15"] / (df_test["sqft_living15"] + 1)
)

In [18]:
lat_center = df_test["lat"].mean()
lon_center = df_test["long"].mean()

df_test["distance_from_center"] = np.sqrt(
    (df_test["lat"] - lat_center) ** 2 +
    (df_test["long"] - lon_center) ** 2
)

df_test["waterfront_view"] = df_test["waterfront"] * df_test["view"]

In [19]:
# Effective age of the property
df_test["house_age"] = 2015 - df_test["yr_built"]

# Renovation indicator
df_test["is_renovated"] = (df_test["yr_renovated"] > 0).astype(int)

# Years since renovation (if any)
df_test["years_since_reno"] = np.where(
    df_test["yr_renovated"] > 0,
    2015 - df_test["yr_renovated"],
    0
)

df_test["year_sale"] = df_test["date"].dt.year
df_test["month_sale"] = df_test["date"].dt.month
df_test["quarter_sale"] = df_test["date"].dt.quarter

In [20]:
df_test.columns

Index(['id', 'date', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot',
       'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above',
       'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long',
       'sqft_living15', 'sqft_lot15', 'above_ground_ratio', 'bath_per_bed',
       'living_to_lot_ratio', 'relative_living_size', 'relative_lot_size',
       'neighborhood_density', 'distance_from_center', 'waterfront_view',
       'house_age', 'is_renovated', 'years_since_reno', 'year_sale',
       'month_sale', 'quarter_sale'],
      dtype='object')

In [21]:
df_test_preprocessed = df_test.drop(columns = ["date"])
df_test_preprocessed.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,relative_lot_size,neighborhood_density,distance_from_center,waterfront_view,house_age,is_renovated,years_since_reno,year_sale,month_sale,quarter_sale
0,2591820310,4,2.25,2070,8893,2.0,0,0,4,8,...,1.154785,3.220410,0.129963,0,29,0,0,2014,10,4
1,7974200820,5,3.00,2900,6730,1.0,0,0,5,8,...,1.070974,2.649937,0.139913,0,38,0,0,2014,8,3
2,7701450110,4,2.50,3770,10893,2.0,0,2,3,11,...,1.124613,2.609809,0.084826,0,18,0,0,2014,8,3
3,9522300010,3,3.50,4560,14608,2.0,0,2,3,12,...,1.026780,3.511725,0.142143,0,25,0,0,2015,3,1
4,9510861140,3,2.50,2550,5376,2.0,0,0,3,9,...,1.327080,1.799200,0.168569,0,11,0,0,2014,7,3


In [22]:
df_test_preprocessed.to_csv(r"C:\Users\ammym\OneDrive\Desktop\CDC\test_preprocessed.csv", index = False)

In [23]:
df_test_preprocessed.shape

(5404, 33)